!{sys.executable} -m pip install numpy


In [28]:
!{sys.executable} -m pip install ipywidgets



In [5]:
import pprint
from collections import defaultdict

class Graph(object):
    """ Graph data structure, undirected by default. """

    def __init__(self, connections=[], directed=False):
        self._graph = defaultdict(set)
        self._directed = directed
        self.add_connections(connections)

    def add_root_with_children(self, node1, connections):
        """ Add connections between a root node and every element of the given list of nodes to graph """

        for node2 in connections:
            self.add(node1, node2)

    def add_connections(self, connections):
        """ Add connections (list of tuple pairs) to graph """

        for node1, node2 in connections:
            self.add(node1, node2)

    def add(self, node1, node2):
        """ Add connection between node1 and node2 """

        self._graph[node1].add(node2)
        if not self._directed:
            self._graph[node2].add(node1)

    def remove(self, node):
        """ Remove all references to node """

        for n, cxns in self._graph.items():  # python3: items(); python2: iteritems()
            try:
                cxns.remove(node)
            except KeyError:
                pass
        try:
            del self._graph[node]
        except KeyError:
            pass

    def is_connected(self, node1, node2):
        """ Is node1 directly connected to node2 """

        return node1 in self._graph and node2 in self._graph[node1]

    def find_path(self, node1, node2, path=[]):
        """ Find any path between node1 and node2 (may not be shortest) """

        path = path + [node1]
        if node1 == node2:
            return path
        if node1 not in self._graph:
            return None
        for node in self._graph[node1]:
            if node not in path:
                new_path = self.find_path(node, node2, path)
                if new_path:
                    return new_path
        return None

    def __str__(self):
        return '{}({})'.format(self.__class__.__name__, dict(self._graph))

    

In [6]:
import re
from bs4 import BeautifulSoup
import json
from anytree import Node, RenderTree
from anytree.search import find
from collections import defaultdict

class SaggerParser:
    def __init__(self):
        self.data= String()
        self.soup= BeautifulSoup()
        self.connectionsDict = defaultdict(set)

    def read_from_xml(self, file,encoding="utf8"):
        '''savegames are in XML format'''
        self.type = 'xml'
        try:
            with open(file, 'r', encoding=encoding) as f:
                self.data = f.read() 
        
        except Exception as e:
            # print(type(e))
            self.data = file

    def createSoup(self):
        self.soup = BeautifulSoup(self.data, "html.parser")

    def generateDict(self):
        for tag in self.soup.findAll(True):
            childList = tag.findChildren()
            if len(childList) > 0:
                self.connectionsDict[tag]=set(childList)

    def generateGraph(self):
        self.graph = Graph()

    def show(self, what='soup', maxDepth=10):
        what=what.lower()
        print(what.upper())
       
        if what=='soup':
            print(self.soup)
        elif what=='pretty':
            print(self.soup.prettify())
        elif what=='structure':
            if (self.type=='xml'):
                for line in self.soup.prettify().split("\n"):
                    match = re.match("(\s*)<([A-Za-z]+)>", line)
                    if match and len(match.group(1)) <= maxDepth:
                        print(len(match.group(1))*'|'+'-' + match.group(2))
        elif what=='connections':
            print(self.connectionsDict)
        elif what=='graph':
            print(self.graph)

In [181]:
import io
sp = SaggerParser()
g=Graph()
try:
    f='CAMP_200_SUEZ_CANAL.sav' #xml file
    # f='' #nothing
    # f = io.StringIO(u"""<a><ab></ab><ad><de></de></ad></a><c></c>""")
    sp.read_from_xml(f)
except FileNotFoundError as e:
    print(e)
    
sp.createSoup()
# sp.generateDict()
# sp.show('pretty')
# sp.show('structure')
# sp.show('connections')
type(sp.data)
        

str

In [88]:
g = Graph()
for i, el  in enumerate(sp.soup.children):
    print(i, el)

0 <a>
<ab></ab>
<ad>
<de></de>
</ad>
</a>
1 

2 <c></c>
3 



In [53]:
from bs4 import BeautifulSoup
import networkx as nx
from collections import defaultdict

class xmlParser():
    def __init__(self, file):
        self.getData(file)
        self.getSoup()
        self.getGraph()
        print(self)

    def getData(self, file):
        try:
            with open(file, 'r', encoding="utf8") as f:
                self.data = f.read() 
        
        except Exception as e:
            self.data = file

        
 
    def getSoup(self):
        self.soup = BeautifulSoup(self.data, "html.parser")
        
    def getGraph(self):
        self.G=nx.Graph()    
        self.connections = []
        # self.edges_dict = defaultdict(set)

        for tag in self.soup.findAll(True):
            childlist = tag.findChildren()
            for _c in childlist:
                edge = (tag.name, _c.name)
                self.connections.append(edge)
                self.G.add_edge(*edge)
            # if len(childlist) > 0:
            #     self.edges_dict[tag]=set(childlist)
        
        # G=nx.Graph(self.edges_dict)    
        # G2=Graph(self.connections)  
        
        # print(G2._graph)
    def plot(self, k=0):
        if k==0:
            nx.draw_networkx(self.G)
        elif k==1:
            nx.draw_circular(self.G)
        elif k==2:
            nx.draw_shell(self.G)
        elif k==3:
            nx.draw_spring(self.G)
        else: 
            nx.draw(self.G)

    def __repr__(self):
        return (nx.info(parser.G))

        

parser = xmlParser('CAMP_200_SUEZ_CANAL.sav')

Graph with 249 nodes and 1054 edges


In [112]:

import matplotlib.pyplot as plt
import matplotlib as mpl
import networkx as nx
import visJS2jupyter.visualizations as visualizations
from visJS2jupyter import visJS_module

G = parser.G
cc = (nx.clustering(G))
degree = G.degree()

# nodes=G.nodes()
nodes_dict=nx.convert.to_dict_of_dicts(G)
edges=G.edges()
edges_dict = dict(zip(edges,range(len(edges))))

# bc = (nx.betweenness_centrality(G))

# nodes = G.nodes()
# edges = G.edges()

# print(type(edges))
# # add a node attributes to color-code by

# # nx.set_node_attributes(G,'clustering_coefficient',cc)
# # nx.set_node_attributes(G,degree,'degree')
# # nx.set_node_attributes(G, nx.betweenness_centrality(G), 'betweenness')

# # nx.set_node_attributes(G,'betweenness_centrality',bc)

# # set node initial positions using networkx's spring_layout function
# pos = nx.spring_layout(G)

# nodes_dict = [{"id":n,
#               "x":pos[n][0]*1000,
#               "y":pos[n][1]*1000} for n in nodes
#               ]
# for i in range(len(edges)):
#     print(edges[i][0])
# # set some network-wide styles
visJS_module.visjs_network(nodes_dict,edges_dict)

KeyError: 0

In [97]:
# https://towardsdatascience.com/processing-xml-in-python-elementtree-c8992941efd2
import xml.etree.ElementTree as ET
tree = ET.parse('CAMP_200_SUEZ_CANAL.sav')

root = tree.getroot()
for child in root:
    print(child.tag, child.attrib, child.get)

root.iter('golan')

flowcontrollers {} <built-in method get of xml.etree.ElementTree.Element object at 0x00000227CC263B00>
globalVariablesSave {} <built-in method get of xml.etree.ElementTree.Element object at 0x00000227CC275440>
popupsContainer {} <built-in method get of xml.etree.ElementTree.Element object at 0x00000227C8913BF0>
unitsBasic {} <built-in method get of xml.etree.ElementTree.Element object at 0x00000227C8913E70>
unitsGroups {} <built-in method get of xml.etree.ElementTree.Element object at 0x00000227C8E84B30>
forts {} <built-in method get of xml.etree.ElementTree.Element object at 0x00000227C49E43B0>
tags {} <built-in method get of xml.etree.ElementTree.Element object at 0x00000227C4AB9260>
DateTime {} <built-in method get of xml.etree.ElementTree.Element object at 0x00000227C4AEBC90>
entrypoints {} <built-in method get of xml.etree.ElementTree.Element object at 0x00000227C4AEBD30>
unlockedDatabaseEntries {} <built-in method get of xml.etree.ElementTree.Element object at 0x00000227C4AEBF60>